# Predicitng tasks from input data

## importing necessary modules

In [2]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


#importing fundtion to get encoder info and perfrom tasks 
from get_finetune_encoder import  get_finetune_input_data


%who



DataLoader	 LabelEncoder	 TensorDataset	 accuracy_score	 f1_score	 get_finetune_input_data	 get_pretrain_input_data	 imaplib	 importlib	 
label_encoder	 ml_code_path	 nn	 np	 optim	 optuna	 os	 pd	 precision_score	 
random	 recall_score	 roc_auc_score	 torch	 


## importing input data

In [3]:
#input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_2'
finetune_save_dir='/home/leilapirhaji/finetune_frozen_layer' 


#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'OS_Event', 'PFS', 'PFS_Event' ]


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_finetune_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((443, 2736), (443, 58), (149, 2736), (149, 58), (149, 2736), (149, 58))

## Predicting binary tasks

#### IMDC BInary

In [4]:
import importlib
import latent_task_predict

importlib.reload(latent_task_predict)
from latent_task_predict import log_reg_multi_class


task = 'IMDC BINARY'

best_val_accuracy, best_val_auc, test_accuracy, test_auc= log_reg_multi_class(task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)


best_val_accuracy, best_val_auc, test_accuracy, test_auc

Test Accuracy with best model: 0.8070
Test AUC with best model: 0.8990


(0.8688524590163934,
 0.9044289044289046,
 0.8070175438596491,
 0.8989769820971867)

## Predicting multi-class tasks 

In [6]:

results = []

# Loop over each task
for task in task_list_cat:
    print(f"Evaluating task: {task}")
    
    # Call the log_reg_multi_class function and get the results
    best_val_accuracy, best_val_auc, test_accuracy, test_auc= log_reg_multi_class(task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)

    # Append the results to the list
    results.append({
        'Task': task,
        'Best Val Accuracy': best_val_accuracy,
        'Best Val AUC': best_val_auc,
        'Test Accuracy': test_accuracy,
        'Test AUC': test_auc
    })

# Convert the list of results to a pandas DataFrame
results_df = pd.DataFrame(results)


results_df

Evaluating task: Benefit BINARY
Test Accuracy with best model: 0.5682
Test AUC with best model: 0.5647
Evaluating task: Nivo Benefit BINARY
Test Accuracy with best model: 0.5490
Test AUC with best model: 0.6815
Evaluating task: MSKCC BINARY
Test Accuracy with best model: 0.8690
Test AUC with best model: 0.9178
Evaluating task: IMDC BINARY
Test Accuracy with best model: 0.8070
Test AUC with best model: 0.8990
Evaluating task: Benefit ORDINAL
Test Accuracy with best model: 0.4094
Test AUC with best model: 0.5617
Evaluating task: MSKCC ORDINAL
Test Accuracy with best model: 0.6242
Test AUC with best model: 0.7793
Evaluating task: IMDC ORDINAL
Test Accuracy with best model: 0.6573
Test AUC with best model: 0.7094
Evaluating task: ORR
Test Accuracy with best model: 0.4295
Test AUC with best model: 0.5789
Evaluating task: Benefit
Test Accuracy with best model: 0.4094
Test AUC with best model: 0.5617
Evaluating task: IMDC
Test Accuracy with best model: 0.6309
Test AUC with best model: 0.6600


,Task,Best Val Accuracy,Best Val AUC,Test Accuracy,Test AUC
0,Benefit BINARY,0.636364,0.617054,0.568182,0.564700
1,Nivo Benefit BINARY,0.588235,0.552469,0.549020,0.681538
2,MSKCC BINARY,0.804878,0.859788,0.869048,0.917833
3,IMDC BINARY,0.868852,0.904429,0.807018,0.898977
4,Benefit ORDINAL,0.442953,0.579309,0.409396,0.561714
5,MSKCC ORDINAL,0.671141,0.793798,0.624161,0.779283
6,IMDC ORDINAL,0.664336,0.761014,0.657343,0.709418
7,ORR,0.523490,0.642596,0.429530,0.578878
8,Benefit,0.442953,0.579309,0.409396,0.561714
9,IMDC,0.644295,0.760153,0.630872,0.660042


## Predicting Survival task

### OS prediction

In [10]:
import importlib

import latent_task_predict_finetune
importlib.reload(latent_task_predict_finetune)
from latent_task_predict_finetune import cox_proportional_hazards


task = 'OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

(0.715054309207187, 0.6761040773601834, {'penalizer': 10, 'l1_ratio': 0})

### PFS prediction 

In [7]:
import importlib

import latent_task_predict
importlib.reload(latent_task_predict)
from latent_task_predict import cox_proportional_hazards


task = 'PFS'
task_event = 'PFS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

(0.6001268767181223, 0.5596460563688005, {'penalizer': 10, 'l1_ratio': 0})